In [ ]:
!pip install spacy
!python -m spacy download en_core_web_sm

!pip install scispacy
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_core_sci_sm-0.4.0.tar.gz

In [ ]:
import spacy
# For a general-purpose spaCy model
# nlp = spacy.load("en_core_web_sm")

# ****************************************************************************************************
# For a SciSpaCy model
import scispacy
nlp = spacy.load("en_core_sci_sm")

# ****************************************************************************************************
text = """Patient shows signs of Type 2 Diabetes Mellitus and hypertension.
Recommend further tests to confirm Chronic Kidney Disease (CKD) stage 2."""

doc = nlp(text)

# ****************************************************************************************************
diagnoses = []
for ent in doc.ents:
    if ent.label_ in ["DISEASE", "ILLNESS"]:  # Adapt these labels based on your model's documentation
        diagnoses.append((ent.start_char, ent.end_char))

# ****************************************************************************************************
cleaned_text = ""
last_idx = 0
for start, end in diagnoses:
    cleaned_text += text[last_idx:start]  # Add text up to the diagnosis
    last_idx = end  # Update the last index to the end of the diagnosis

cleaned_text += text[last_idx:]  # Add any remaining text after the last diagnosis


In [ ]:
from transformers import AutoModelForTokenClassification, AutoTokenizer, pipeline

def split_text(text, chunk_size=512):
    # Splits the text into chunks of up to `chunk_size` characters
    return [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

# Initialize the model and tokenizer
model_name = "emilyalsentzer/Bio_ClinicalBERT"  # Replace with your model's name
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer)

# Your original clinical note text
text = clin_notes[0]  # Assuming clin_notes[0] is a string containing the clinical note

# Split the text into manageable chunks
chunks = split_text(text, 128)  # You might adjust chunk_size based on your needs

# Process each chunk and collect results
all_results = []
for chunk in chunks:
    chunk_results = ner_pipeline(chunk)
    all_results.extend(chunk_results)

# Filter results to get only diagnoses (replace 'Your_Diagnosis_Entity_Label' with actual label)
diagnoses = [result for result in all_results if result['entity'] == 'Your_Diagnosis_Entity_Label']

# Print out the diagnoses found
for diagnosis in diagnoses:
    print(f"Diagnosis: {diagnosis['word']}, Score: {diagnosis['score']}")

